In [1]:

from catboost import CatBoostRegressor
from tpot import TPOTRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder,LabelEncoder
from xgboost import XGBRegressor
from tensorflow.keras.layers import Normalization
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error as mae


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/intra-aims-ml-challenge/test_set.csv
/kaggle/input/intra-aims-ml-challenge/submission.csv
/kaggle/input/intra-aims-ml-challenge/train_set.csv


In [2]:
train=pd.read_csv("/kaggle/input/intra-aims-ml-challenge/train_set.csv")
test1=pd.read_csv("/kaggle/input/intra-aims-ml-challenge/test_set.csv")
test=pd.read_csv("/kaggle/input/intra-aims-ml-challenge/test_set.csv")


In [3]:
train.describe(include='object')


,Product_Code,Fat_Category,Category_Type,Store_Code,Store_Size,Location_Class,Store_Category
count,7750,7750,7750,7750,5565,7750,7750
unique,1558,2,16,10,3,3,4
top,FDW13,Reduced,Produce,OUT049,Standard,Rural,Superstore A
freq,10,5027,1114,854,2537,3043,5076


In [4]:
X=train[['Product_Code','Product_Weight','Fat_Category','Visibility_Score','Category_Type','Retail_Price','Store_Code','Year_Opened','Store_Size','Location_Class','Store_Category']]
Y=train.Total_Sales


In [5]:
X.head()

,Product_Code,Product_Weight,Fat_Category,Visibility_Score,Category_Type,Retail_Price,Store_Code,Year_Opened,Store_Size,Location_Class,Store_Category
0,FDI28,14.300,Reduced,0.026300,Frozen Foods,79.4302,OUT013,1987,Spacious,Rural,Superstore A
1,NCM17,7.930,Reduced,0.071136,Health and Hygiene,42.7086,OUT046,1997,Compact,Urban,Superstore A
2,FDC14,14.500,Standard,0.041313,Canned,42.0454,OUT049,1999,Standard,Urban,Superstore A
3,DRC36,NaN,Standard,0.044767,Beverages,173.7054,OUT027,1985,Standard,Rural,Superstore C
4,FDS27,10.195,Standard,0.012456,Proteins,197.5110,OUT035,2004,Compact,Suburban,Superstore A


In [6]:
X['Year_Opened'] = 2024 - X['Year_Opened']

cat_cols = ['Product_Code','Fat_Category','Category_Type','Store_Code','Store_Size','Location_Class', 'Store_Category']  
num_cols = ['Product_Weight', 'Visibility_Score', 'Retail_Price', 'Year_Opened']  # Example numerical columns


cat_tf1 = Pipeline(steps=[
     
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('imputer', SimpleImputer(strategy='mean')) 
])




scaler = StandardScaler()

from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()


X_scaled = scaler.fit_transform(X[['Visibility_Score']])

print(X_scaled)


label_encoder = LabelEncoder()
X['Fat_Category'] = label_encoder.fit_transform(X['Fat_Category'])
X['Store_Size'] = label_encoder.fit_transform(X['Store_Size'])


freq_encoding = X['Product_Code'].value_counts().to_dict()
X['Product_Code'] = X['Product_Code'].map(freq_encoding)

X['Visibility_Score'].fillna(X['Visibility_Score'].median(), inplace=True)


X['Product_Weight'] = X.groupby('Category_Type')['Product_Weight'].transform(lambda x: x.fillna(x.mean()))


preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', cat_tf1, cat_cols),  
        ('scaler', scaler, num_cols)  
    ]
)


X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)


[[-0.41005703]
 [ 0.2541865 ]
 [-0.18763439]
 ...
 [ 0.50691241]
 [-0.27621292]
 [ 0.72507946]]


/tmp/ipykernel_17/2146704632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Year_Opened'] = 2024 - X['Year_Opened']
/tmp/ipykernel_17/2146704632.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Fat_Category'] = label_encoder.fit_transform(X['Fat_Category'])
/tmp/ipykernel_17/2146704632.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [7]:
model = CatBoostRegressor(
    iterations=2400, 
    l2_leaf_reg=5,
    learning_rate=0.0035,            
    depth=5,
    early_stopping_rounds=50,                      
    loss_function="RMSE",         
    verbose=70                  
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  
    ('model', model)   ])             

pipeline.fit(X_train, Y_train)

Y_pred = pipeline.predict(X_valid)

# Step 11: Calculate errors
print("Mean Squared Error = ", mse(Y_valid, Y_pred))
print("Mean Absolute Error = ", mae(Y_valid, Y_pred), "\n")

0:	learn: 1697.8140603	total: 55ms	remaining: 2m 11s
70:	learn: 1505.7356388	total: 171ms	remaining: 5.61s
140:	learn: 1367.3418019	total: 290ms	remaining: 4.64s
210:	learn: 1269.8280950	total: 404ms	remaining: 4.19s
280:	learn: 1203.3028021	total: 520ms	remaining: 3.92s
350:	learn: 1157.3507838	total: 632ms	remaining: 3.69s
420:	learn: 1126.8443833	total: 746ms	remaining: 3.51s
490:	learn: 1105.9638743	total: 858ms	remaining: 3.34s
560:	learn: 1091.6797215	total: 971ms	remaining: 3.18s
630:	learn: 1081.6221194	total: 1.08s	remaining: 3.04s
700:	learn: 1074.8792512	total: 1.2s	remaining: 2.9s
770:	learn: 1069.9204177	total: 1.3s	remaining: 2.76s
840:	learn: 1066.1705404	total: 1.42s	remaining: 2.63s
910:	learn: 1063.2743875	total: 1.52s	remaining: 2.49s
980:	learn: 1060.6830433	total: 1.64s	remaining: 2.38s
1050:	learn: 1058.6162199	total: 1.75s	remaining: 2.25s
1120:	learn: 1056.8960912	total: 1.87s	remaining: 2.13s
1190:	learn: 1055.2080136	total: 1.98s	remaining: 2.01s
1260:	learn: 

In [8]:
test["Year_Opened"]=2024-test["Year_Opened"]

test_pred=pipeline.predict(test)

In [9]:
output=pd.DataFrame({'Id':test1.Id,'Total_Sales':test_pred})
output.to_csv('AnugyaAIMSSubmission.csv',index=False)